# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"


In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
from __future__ import division, print_function

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import numpy as np

from IPython.core.display import HTML, Image

In [3]:
r = requests.get(URL)
indeed = BeautifulSoup(r.content, "lxml")


In [4]:
col_list = ['location', 'title', 'company', 'salary', 'description']

def prime_time(soup):
    location_list = []
    role_title_list = []
    company_list = []
    salaries = []
    summaries = []

    for t in soup.findAll("div", class_='result'):
        location = t.find('span', class_='location').text # WORKS!
        location_list.append(location)
        title = t.find('a')['title'] # WORKS!
        role_title_list.append(title)
        company_name = t.find('span', class_='company').text
        company_name = company_name.strip('\n')
        company_list.append(company_name)
        salary = t.find('nobr' or 'sjcl')   #('nobr').next_sibling sjcl
        salaries.append(salary)
        summary = t.find('span', class_="summary").text
        summaries.append(summary)
    
    results = zip(location_list, role_title_list, company_list, salaries, summaries)
    results = pd.DataFrame(results)
    results.columns = col_list
    return(results)

# indeed_results = prime_time(indeed)
# indeed_output = pd.DataFrame(indeed_results)
# indeed_output.head()



Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above
- Include any other features you may want to use later (e.g. summary, #of reviews...)

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Indeed.com only has salary information for an estimated 20% of job postings. You may want to add other cities to the list below to gather more data. 
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
#main scraping piece
import numpy as np

url_template_first = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}"
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"

max_results_per_city = 100
col_list = ['location', 'title', 'company', 'salary', 'description']
sample_vals = [np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]

d = dict.fromkeys(col_list, sample_vals)
results = pd.DataFrame(d)   # trying to make the columns the same so I can merge later


for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Atlanta','Boston','Berkeley',\
                 'Oakland', 'Raleigh', 'Brooklyn','Queens', 'St+Louis','Asheville','Portland','Maine',\
                 'Baltimore', 'Boulder', 'Denver', 'Reno', 'Des Moines', 'Cleveland',  'Detroit', 'Kansas+City',\
                 'Dallas', 'Washington+DC', 'Nashville', 'Seattle', 'Salt+Lake+City', 'Sacramento', 'Minneapolis', 'St+Paul']): 
    for start in range(0, max_results_per_city, 10):
        foo = requests.get(url_template.format(city, start))
        indeed = BeautifulSoup(foo.content, "lxml")
        indeed_results = prime_time(indeed)
        indeed_results.columns = col_list
        
        results = pd.concat([indeed_results, results], ignore_index=True)
    first = requests.get(url_template_first.format(city)) #GETS THE FIRST PAGE OF RESULTS, MISSED
    first_indeed = BeautifulSoup(first.content, "lxml")
    first_indeed_results = prime_time(first_indeed)
    first_indeed_results.columns = col_list
    results = pd.concat([first_indeed_results, results], ignore_index=True)
results.head(50)



,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",None,Senior Research Analyst
1,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,None,Biomedical Laboratory Officer
2,AdvancePierre Foods,Lead multiple complex development projects usi...,"Portland, ME",None,"Food Scientist-Poultry Protein, Culinary Conce..."
3,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",None,Big Data Engineer
4,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",None,Statistician
5,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",None,Scientist I (Research Assoc)
6,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME","<nobr>$120,000 a year</nobr>",Data Analyst II/III
7,AdvancePierre Foods,\nLead multiple complex development projects u...,"Portland, ME",None,"Food Scientist-Poultry Protein, Culinary Conce..."
8,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",None,Senior Research Analyst
9,IDEXX Laboratories,\nSeeking an Associate Scientist to join a gro...,"Westbrook, ME 04092",None,Associate Scientist/Engineer – Systems Chemistry


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now 2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [6]:
import numpy as np
            
# results['annual_salary'] = results['salary']

results_nan = results.replace({None:np.NaN})
results = results_nan
results.head(10)
            

,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst
1,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer
2,AdvancePierre Foods,Lead multiple complex development projects usi...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce..."
3,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer
4,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician
5,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",NaN,Scientist I (Research Assoc)
6,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME","<nobr>$120,000 a year</nobr>",Data Analyst II/III
7,AdvancePierre Foods,\nLead multiple complex development projects u...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce..."
8,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",NaN,Senior Research Analyst
9,IDEXX Laboratories,\nSeeking an Associate Scientist to join a gro...,"Westbrook, ME 04092",NaN,Associate Scientist/Engineer – Systems Chemistry


In [7]:
def salary_clean(string):
    if type(string) == float:
        return np.NaN
    elif string == np.NaN:
        return np.NaN
    else:
        try:
            return((string.split("<nobr>")[1]).split("</nobr>")[0])
        except (TypeError, IndexError): 
            return np.NaN

for b in range(len(results['salary'])):
    results['salary'][b] = salary_clean(str(results['salary'][b]))
    

results.head(15)

,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst
1,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer
2,AdvancePierre Foods,Lead multiple complex development projects usi...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce..."
3,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer
4,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician
5,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",NaN,Scientist I (Research Assoc)
6,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME","$120,000 a year",Data Analyst II/III
7,AdvancePierre Foods,\nLead multiple complex development projects u...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce..."
8,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",NaN,Senior Research Analyst
9,IDEXX Laboratories,\nSeeking an Associate Scientist to join a gro...,"Westbrook, ME 04092",NaN,Associate Scientist/Engineer – Systems Chemistry


In [8]:
# print(results['salary'].isnull().value_counts()) #only 62 salaries in here it appears


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
import re
import string 

def salarizer(salary):  
    '''remove all non-numeric characters from salary strings for non-null vals '''
    if pd.isnull(salary) == False:
        salary = salary.translate(None, "$|anmonthhurye,dwk ")
        return(salary) 
    else:
        return np.NaN

    


def salary_average(salary): 
    '''if a salary range, take the average, otherwise return the number'''
    float_sal_list = []
    try:
        salary_str = str(salary)
        sal_list = str.split(salary_str, "-")
        for item in sal_list:
            try:
                float_sal_list.append(float(item))
            except ValueError:
                return salary
        if len(float_sal_list) > 1:
            avg_sal = sum(float_sal_list)/float(len(float_sal_list))
            return(avg_sal)
        else:
            return float(salary)
    except IndexError:
        return np.NaN
    

    
for hannah in range(len(results['salary'])):
    results['salary'][hannah] = salarizer(results['salary'][hannah])
    results['salary'][hannah] = salary_average(results['salary'][hannah])       
    

In [10]:



results['hourly_salary'] = results['salary'].apply(lambda x: x if float(x) < 200 else np.nan) 
results['monthly_salary'] = results['salary'].apply(lambda x: x if float(x) < 10000 else np.nan)
results['annual_salary'] = results['salary'].apply(lambda x: x if float(x) > 10000 else np.nan)



# #categorize hourly, monthly, salaries ---- this is far and away the most time-intensive part of the script
###REWROTE THIS FUNCTION BELOW ABOVE FOR PERFORMANCE REASONS

# results['monthly_salary'] = np.NaN
# results['annual_salary'] = np.NaN

# for where, number in enumerate(results['salary']):
#     if number < 200: #max of hourly is less
#         results['hourly_salary'][where] = number
#     elif number <= 10000: #max of monthly is less than this 
#         results['monthly_salary'][where] = number
#     else:
#         results['annual_salary'][where] = number


### Save your results as a CSV

In [11]:
#remove duplicates! Addition of Oakland, Berkeley, SF etc... will lead to dupes

results.drop_duplicates(inplace=1)

In [12]:
results.to_csv("indeed_ds_data30_dedupe.csv", encoding='utf8')

#at this point, there are 158 annual salaries in my dataset
#https://docs.google.com/spreadsheets/d/1Luw7rFbYnh31zq2ZsCJ0pGrJQ4FnVfsHv1RRTUmGGuw/edit?usp=sharing

In [13]:
results.head() 

,company,description,location,salary,title,hourly_salary,monthly_salary,annual_salary
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst,NaN,NaN,NaN
1,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer,NaN,NaN,NaN
2,AdvancePierre Foods,Lead multiple complex development projects usi...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce...",NaN,NaN,NaN
3,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer,NaN,NaN,NaN
4,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician,NaN,NaN,NaN


In [14]:
indeeder = results
indeeder.head()

,company,description,location,salary,title,hourly_salary,monthly_salary,annual_salary
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst,NaN,NaN,NaN
1,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer,NaN,NaN,NaN
2,AdvancePierre Foods,Lead multiple complex development projects usi...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce...",NaN,NaN,NaN
3,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer,NaN,NaN,NaN
4,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician,NaN,NaN,NaN


In [15]:
indeeder.describe()

,hourly_salary,monthly_salary,annual_salary
count,41.000000,73.000000,150.000000
mean,30.894756,2332.214863,106758.760000
std,16.687505,3075.281498,41533.058353
min,11.965000,11.965000,30000.000000
25%,20.000000,24.185000,70250.000000
50%,25.000000,60.000000,108750.000000
75%,34.000000,4313.500000,136875.000000
max,75.000000,9350.000000,240000.000000


### Beginning Analysis Phase of Project

In [16]:
print(type(indeeder['annual_salary'][4]),type(indeeder['annual_salary'][5]))

<type 'numpy.float64'> <type 'numpy.float64'>


In [17]:
# annual_avg_sal = np.mean(indeed['annual_salary']) #not working, typeerror but can't figure it out

def meaner(series):
    '''finds the mean amidst nans'''
    mean_list = []
    for example in series:
        if example != np.NaN:
            try:
                mean_list.append(float(example))
            except (TypeError, ValueError):
                continue
        else:
            continue
    return(np.nanmean(mean_list))

In [18]:
indeeder['salary'] = (indeeder['monthly_salary']*12)
indeeder['salary'] = (indeeder['hourly_salary']*12*5*9)#12 month, 5 days/ week, 9 hours/day
avg_pay = meaner(indeeder['salary'])
avg_annual_salary = float(meaner(indeeder['annual_salary']))

print("all data:", avg_pay,"annual:", avg_annual_salary)
#all data: 17248.0764706 annual: 107525.268987
#conclusion is: only use annual pay salaries, or avg_annual_salary

all data: 16683.1682927 annual: 106758.76


In [19]:
# indeeder['above_average_salary'] = indeeder['annual_salary'].apply(lambda x: 1 if x > avg_annual_salary else continue)
# indeeder['above_average_salary'] = indeeder['annual_salary'].apply(lambda x: 0 if x < avg_annual_salary else continue)

def checker(value):
    try:
        if value > avg_annual_salary:
            return 1
        elif value < avg_annual_salary:
            return 0
    except ValueError:
        return 'error'

indeeder['above_average_salary'] = indeeder['annual_salary'].to_frame().applymap(checker)

In [20]:
#create target

#######THIS WAS REPLACED BY FUNCTION ABOVE USING APPLYMAP
# indeeder['above_average_sal'] = np.NaN
# for ind, val in enumerate(indeeder['annual_salary']):
#     if val != np.NaN:
#         try:
#             if float(val) >  107525.26:
#                 indeeder['above_average_sal'][ind] = 1
#             elif float(val) < 107525.26:
#                 indeeder['above_average_sal'][ind] = 0
#             else:
#                 indeeder['above_average_sal'][ind] = 'error'
#         except ValueError:
#             continue
#         else:
#             continue
#     else:
#         continue


In [21]:

skill_list = ['python', 'r', 'learning', 'SQL', 'visualiz', 'mining', 'mung', 'clean', 'scikit-learn']
big_list = ['big', 'advanced', 'volume', 'hadoop', 'spark', 'engineer', 'large']
senior_list = ['senior', 'sr', 'Sr.', 'lead', 'principle', 'chief', 'specialist']

indeeder['skill_count'] = 0  
indeeder['senior_count'] = 0 
indeeder['big_count'] = 0 
indeeder['constant'] = 1
indeeder = indeeder.reset_index()
        
        
for z in range(len(indeeder['description'])):
    try:
        for h in str(indeeder['description'][z]).split():  
            if any(h in s for s in skill_list):
                indeeder.ix[z,'skill_count'] += 1
        for u in indeeder['description'][z].split():  
            if any(u in k for k in senior_list):
                indeeder.ix[z,'senior_count'] += 1
        for d in indeeder['description'][z].split():
            if any(d in q for q in big_list):
                indeeder.ix[z,'big_count'] += 1
    except (ValueError, AttributeError):
        continue
            
indeeder.head(30)

#looks like this starts dropping values around 15th row, also rows 29-39

,index,company,description,location,salary,title,hourly_salary,monthly_salary,annual_salary,above_average_salary,skill_count,senior_count,big_count,constant
0,0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst,NaN,NaN,NaN,NaN,0,0,0,1
1,1,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer,NaN,NaN,NaN,NaN,2,1,2,1
2,2,AdvancePierre Foods,Lead multiple complex development projects usi...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce...",NaN,NaN,NaN,NaN,0,0,0,1
3,3,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer,NaN,NaN,NaN,NaN,0,0,0,1
4,4,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician,NaN,NaN,NaN,NaN,1,1,1,1
5,5,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",NaN,Scientist I (Research Assoc),NaN,NaN,NaN,NaN,3,3,3,1
6,6,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME",NaN,Data Analyst II/III,NaN,NaN,120000.0,1,0,0,0,1
7,7,AdvancePierre Foods,\nLead multiple complex development projects u...,"Portland, ME",NaN,"Food Scientist-Poultry Protein, Culinary Conce...",NaN,NaN,NaN,NaN,0,0,0,1
8,8,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",NaN,Senior Research Analyst,NaN,NaN,NaN,NaN,0,0,0,1
9,9,IDEXX Laboratories,\nSeeking an Associate Scientist to join a gro...,"Westbrook, ME 04092",NaN,Associate Scientist/Engineer – Systems Chemistry,NaN,NaN,NaN,NaN,2,1,2,1


In [22]:
# before creating subsample with valid y values, checking shape/size
indeeder['annual_salary'].value_counts()

150000.0    12
52842.0      6
110000.0     6
155000.0     5
130000.0     4
140000.0     4
120000.0     4
100000.0     4
95000.0      4
135000.0     4
125000.0     4
125221.0     3
85000.0      3
105000.0     3
78483.0      2
165000.0     2
127500.0     2
132500.0     2
70000.0      2
112500.0     2
94531.0      2
200000.0     2
75000.0      2
65000.0      2
190000.0     2
76856.5      1
46332.0      1
124000.0     1
30000.0      1
50508.0      1
            ..
65643.5      1
105969.5     1
138300.0     1
77192.0      1
47406.0      1
93645.0      1
65060.0      1
77500.0      1
61183.0      1
180000.0     1
110445.0     1
240000.0     1
137500.0     1
145000.0     1
60934.5      1
50794.0      1
87792.0      1
42840.0      1
90000.0      1
49628.5      1
78195.5      1
58175.5      1
51438.0      1
124297.5     1
54963.5      1
50439.5      1
68983.5      1
170000.0     1
116850.0     1
82964.5      1
Name: annual_salary, dtype: int64

Baseline accuracy in the model should be 50%, as we're going to compare each opportunity to the mean value found in the dataset, meaning there are only two options: higher or lower. If we cannot create a model that outperforms a coinflip, we have failed miserably. 

In [23]:
import statsmodels.api as sm
import pandas as pd
import pylab as pl
import numpy as np
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score

vals = [0,1]
indeeder_subsamp = indeeder.loc[indeeder['above_average_salary'].isin(vals)]
y = indeeder_subsamp['above_average_salary']
y.head()
#only 5 values to train on? Can't figure out where I'm losing all this data


6     1
10    0
11    0
25    0
31    0
Name: above_average_salary, dtype: float64

In [24]:
#WILL RETURN TO THIS, TRYING TO UNDERSTAND DROPPED DATA

# city_list = ['SF', 'San Francisco', 'Francisco', 'New York', 'York', 'NY', 'NYC']
# indeeder['big_city'] = 0

# for gindex, gvalue in enumerate(indeeder['location']):
#     if any(str(gvalue).split()) in city_list:
#         indeeder['big_city'][gindex] = 1


# X_lame = pd.get_dummies(indeeder.loc['big_city', 'constant'])

In [25]:
#most basic logistic regression
indeeder.describe()

,index,salary,hourly_salary,monthly_salary,annual_salary,above_average_salary,skill_count,senior_count,big_count,constant
count,2301.000000,41.000000,41.000000,73.000000,150.000000,150.000000,2301.000000,2301.000000,2301.000000,2301
mean,2321.040852,16683.168293,30.894756,2332.214863,106758.760000,0.506667,0.960017,0.869187,0.731421,1
std,1414.397821,9011.252527,16.687505,3075.281498,41533.058353,0.501630,1.094318,1.032857,0.901694,0
min,0.000000,6461.100000,11.965000,11.965000,30000.000000,0.000000,0.000000,0.000000,0.000000,1
25%,1158.000000,10800.000000,20.000000,24.185000,70250.000000,0.000000,0.000000,0.000000,0.000000,1
50%,2075.000000,13500.000000,25.000000,60.000000,108750.000000,1.000000,1.000000,1.000000,0.000000,1
75%,3427.000000,18360.000000,34.000000,4313.500000,136875.000000,1.000000,2.000000,1.000000,1.000000,1
max,5046.000000,40500.000000,75.000000,9350.000000,240000.000000,1.000000,6.000000,6.000000,5.000000,1


In [26]:
dumdums = pd.get_dummies(indeeder_subsamp[['senior_count', 'skill_count', 'big_count', 'constant']])#will add 'big_city' later

In [27]:
X = dumdums
X.head()

,senior_count,skill_count,big_count,constant
6,0,0,0,1
10,0,0,0,1
11,1,1,0,1
25,1,2,1,1
31,1,1,1,1


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)

In [29]:
lr_basic = LogisticRegression(solver='liblinear') #can i take away penalty? make sure this is statsmodels
lr_basic_model = lr_basic.fit(X_train, y_train)
lr_basic_ypred = lr_basic_model.predict(X_test)

lr_basic_cm = confusion_matrix(y_test, lr_basic_ypred, labels=lr_basic.classes_)
lr_basic_cm_df = pd.DataFrame(lr_basic_cm, columns=lr_basic.classes_, index=lr_basic.classes_) #lr_l1_SF is instance of model
lr_basic_cm_df
#can't print classification report? Taking too long to run
#appears my model is about as well as average; precision = 11/19, recall = 11/27 

,0.0,1.0
0,15,8
1,16,11


In [ ]:
# generic curve plotting function
import matplotlib.pyplot as plt
from sklearn import metrics
def auc_plotting_function(rate1, rate2, rate1_name, rate2_name, curve_name):
    AUC = metrics.auc(rate1, rate2)
    # Plot of a ROC curve for class 1 (has_cancer)
    plt.figure(figsize=[11,9])
    plt.plot(rate1, rate2, label=curve_name + ' (area = %0.2f)' % AUC, linewidth=4)
    plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(rate1_name, fontsize=18)
    plt.ylabel(rate2_name, fontsize=18)
    plt.title(curve_name + ' for salary above average', fontsize=18)
    plt.legend(loc="lower right")
    plt.show()

# plot receiving operator characteristic curve
def plot_roc(y_true, y_score):
    fpr, tpr, _ = metrics.roc_curve(y_true, y_score)
    auc_plotting_function(fpr, tpr, 'False Positive Rate', 'True Positive Rate', 'ROC')

In [ ]:
y_score_basic = lr_basic.decision_function(X_test)
plot_roc(y_test, y_score_basic)


In [ ]:
classification_report(y_test, lr_basic_ypred, labels=lr_basic.classes_)

In [ ]:
cross_val_score(lr_basic, X, y, cv=10).mean()

In [ ]:
#Lasso
lr_l1 = LogisticRegression(penalty='l1', solver='liblinear') #can i take away penalty? make sure this is statsmodels
lr_l1_model = lr_l1.fit(X_train, y_train)
lr_l1_ypred = lr_l1_model.predict(X_test)

lr_l1_cm = confusion_matrix(y_test, lr_l1_ypred, labels=lr_l1.classes_)
lr_l1_cm_df = pd.DataFrame(lr_l1_cm, columns=lr_basic.classes_, index=lr_l1.classes_) #lr_l1_SF is instance of model
lr_l1_cm_df

In [ ]:
print(classification_report(y_test, lr_l1_ypred, labels=lr_l1.classes_))

In [ ]:
cross_val_score(lr_l1, X, y, cv=10).mean()

In [ ]:
Y_score_l1 = lr_l1.decision_function(X_test)
plot_roc(Y_test, Y_score_l1)

In [ ]:
#Ridge
lr_l2 = LogisticRegression(penalty='l2', solver='liblinear') #can i take away penalty? make sure this is statsmodels
lr_l2_model = lr_l2.fit(X_train, y_train)
lr_l2_ypred = lr_l2_model.predict(X_test)

lr_l2_cm = confusion_matrix(y_test, lr_l2_ypred, labels=lr_l2.classes_)
lr_l2_cm_df = pd.DataFrame(lr_lr_l2_cm, columns=lr_l2.classes_, index=lr_l2.classes_) #lr_l1_SF is instance of model
lr_l2_cm_df

In [ ]:
print(classification_report(y_test, lr_l2_ypred, labels=lr_l2.classes_))

In [ ]:
cross_val_score(lr_l2, X, y, cv=10).mean()

In [ ]:
Y_score_l2 = lr_l1.decision_function(X_test)
plot_roc(Y_test, Y_score_l2)

In [ ]:
#grid search optimization
logreg = LogisticRegression(solver='liblinear')
C_vals = [0.0001, 0.001, 0.01, 0.1, .15, .25, .275, .33, 0.5, .66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals}, verbose=False, cv=15)
gs.fit(X, y)

In [ ]:
gs.best_params_

In [ ]:
logreg = LogisticRegression(C=gs.best_params_['C'], penalty=gs.best_params_['penalty'])
cv_model = logreg.fit(X_train, y_train)

In [ ]:
cv_pred = cv_model.predict(X_test)

In [ ]:
cm3 = confusion_matrix(y_test, cv_pred, labels=logreg.classes_)
cm3 = pd.DataFrame(cm3, columns=logreg.classes_, index=logreg.classes_)

In [ ]:
cm3

In [ ]:
print(classification_report(y_test, cv_pred, labels=logreg.classes_))

In [ ]:
Y_score_gs = logreg.decision_function(X_test)
plot_roc(y_test, Y_score_gs)